In [1]:
import numpy as np
import cv2
min_confidence = 0.2

In [2]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))
net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt.txt", "MobileNetSSD_deploy.caffemodel")

In [3]:
def serial(img,num):
    image = cv2.imread(img)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

    #load image in model
    net.setInput(blob)
    detections = net.forward()

    #look for match in image
    for i in np.arange(0, detections.shape[2]):
        
        #extract confidence from detected object
        confidence = detections[0, 0, i, 2]

        #filter out low confidence objects
        if confidence > min_confidence:
            
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            
            (startX, startY, endX, endY) = box.astype("int")

            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
#             print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)
    
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # show the output image
    cv2.imwrite("./output/{}_output.jpg".format(num), image)

In [4]:
%%time
size = 6000
print("Serial Implementation of {} images: ".format(size))
for i in range(size):
    serial("./images/image_{}.jpg".format(i),i)

Serial Implementation of 6000 images: 
CPU times: user 17min 11s, sys: 1min 38s, total: 18min 49s
Wall time: 5min 24s
